## Märksõnade lisamine eeltreenitud Word2Vec mudeli abil

Isetreenitud Word2Vec mudeli abil sai eelnevalt teemasid ilmestavate märksõnade hulka täiendatud, kuid siiski suhteliselt vähe: suur hulk sõnadest, mida mudel sarnasteks pidas, olid täiesti sobimatud. See on ootuspärane, sest treeningandmete hulk ei olnud eriti suur. Paremaid tulemusi võiks saada tunduvalt suuremal andmehulgal juba eeltreenitud mudeliga, mida järgnevalt implementeeritaksegi, et suurendada veelgi märksõnade hulka.
Võrreldes oma andmetel treenitud mudeliga on eeltreenitud mudeli puuduseks see, et selle treenimiseks kasutatud sõnavara ei pruugi siinsetes andmetes leiduvaga kattuda ning selle pakutavad märksõnad ei pruugi üldse siinses andmestikus esineda. Selle probleemi lahendamiseks filtreeritakse tulemusi: alles jäetakse vaid verbid, mis esinevad ka siinses andmestikus.

In [89]:
import gensim
from estnltk import Text
import sqlite3
import csv
from estnltk.converters import json_to_text

### I Word2Vec mudel

Eeltreenitud Word2Vec mudeleid on mitmeid (täpsem info https://github.com/estnltk/word2vec-models/, mudelid on treenitud optimaalsete parameetritega). Siin võetakse kasutusele neist üks.

In [8]:
model = gensim.models.KeyedVectors.load_word2vec_format('lemmas.sg.s200.w2v.bin', binary=True)

INFO:keyedvectors.py:2047: loading projection weights from lemmas.sg.s200.w2v.bin
INFO:utils.py:448: KeyedVectors lifecycle event {'msg': 'loaded (441391, 200) matrix of type float32 from lemmas.sg.s200.w2v.bin', 'binary': True, 'encoding': 'utf8', 'datetime': '2023-12-01T10:13:40.716356', 'gensim': '4.3.2', 'python': '3.10.11 | packaged by Anaconda, Inc. | (main, Apr 20 2023, 18:56:50) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'load_word2vec_format'}


Üks sõna ja selle kümme kõige sarnasemat sõna:

In [6]:
model.most_similar('häbistama')

[('häbistama|häbistanud', 0.6818853616714478),
 ('mõnitama', 0.6587278246879578),
 ('vaenama', 0.6524444818496704),
 ('mustama|mustanud', 0.6495568156242371),
 ('blameerima', 0.6478762626647949),
 ('käseva|käsevad', 0.6470065712928772),
 ('naeruvääristama', 0.6439527869224548),
 ('teotama', 0.6419845819473267),
 ('šantažeerima', 0.636863648891449),
 ('laimama|laimatud', 0.6340272426605225)]

On näha, et suur osa vastetest on tõesti sarnase tähendusega sõnale "häbistama". Järgnevalt loetakse CSV-failist siiani kogutud märksõnad ning leitakse mudeli abil neile kõigile kümme sarnaseimat sõna (filtreerides välja korduvused). Seejärel filtreeritakse tulemusi uuesti, jättes alles vaid sõnad, mis on verbid ja esinevad siinses andmestikus. Viimaks vaadatakse järelejäänud sõnad üle manuaalselt ning lisatakse sobivate teemade alla märksõnastikus.

### II Olemasolevatele märksõnadele sarnaste sõnade leidmine

In [65]:
# tühjad järjendid iga teema jaoks
vaidl_erim = []
valimised = []
valetamine = []
valitsemine = []

In [66]:
# olemasolevad märksõnad lotekse järjenditesse
with open('thematic_keywords.csv', encoding='UTF-8') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        if row['class'] == 'vaidlused ja erimeelsused':
            vaidl_erim.append(row['lemma'].strip("',"))
        elif row['class'] == 'valimised':
            valimised.append(row['lemma'].strip("',"))
        elif row['class'] == 'valetamine':
            valetamine.append(row['lemma'].strip("',"))
        elif row['class'] == 'valitsemine':
            valitsemine.append(row['lemma'].strip("',"))

In [128]:
print(valetamine)

['valetama', 'varjama', 'mustama', 'kuritarvitama', 'manipuleerima']


In [129]:
# tühjad järjendid Word2Vec mudeli abil leitavate sõnade jaoks
vaidl_erim_w2v = []
valimised_w2v = []
valetamine_w2v = []
valitsemine_w2v = []

In [130]:
for word in vaidl_erim:
    vaidl_erim_w2v.append([w[0] for w in model.most_similar(word)])
    
for word in valimised:
    valimised_w2v.append([w[0] for w in model.most_similar(word)])
    
for word in valetamine:
    valetamine_w2v.append([w[0] for w in model.most_similar(word)])
    
for word in valitsemine:
    valitsemine_w2v.append([w[0] for w in model.most_similar(word)])

In [131]:
print(valetamine_w2v)

[['vassima', 'valetama|valetanud', 'silmakirjatsema', 'luiskama', 'vassima|vassinud', 'valetama|valetatud', 'kingoks', 'luisanud|luiskama', 'laimama|laimatud', 'valetamine'], ['varjama|varjanud', 'varjav', 'peitma', 'paljastama', 'varjamine', 'paljastama|paljastanud', 'reetma', 'peitev', 'varjatav', 'varjamata'], ['tümitama|tümitanud', 'kuldpagun', 'ratsutav', 'leinarõivas', 'silmalapp', 'krillima', 'valelik', 'poseerinu', 'hõikav', 'uusüritaja'], ['kuritarvitama|kuritarvitanud', 'kuritarvitamine', 'ametiseisund', '1993.-1998.', 'kuritarvitanu', 'kuritarvitav', 'teenistushuvi', 'petma', 'kurjasti', 'ameti-seisund'], ['manipuleerimine', 'omakasupüüdlikult', 'manipuleeritavus', 'manipuleerija', 'desorienteerima', 'shantazheerima', 'manipuleeritav', 'manipuleeriv', 'manipulatsioon', 'mittemõtlev']]


In [132]:
# kahedimensioonilistest listidest saavad ühedimensioonilised
vaidl_erim_w2v = [w for wrds in vaidl_erim_w2v for w in wrds]
valimised_w2v = [w for wrds in valimised_w2v for w in wrds]
valetamine_w2v = [w for wrds in valetamine_w2v for w in wrds]
valitsemine_w2v = [w for wrds in valitsemine_w2v for w in wrds]

In [133]:
print(valetamine_w2v)

['vassima', 'valetama|valetanud', 'silmakirjatsema', 'luiskama', 'vassima|vassinud', 'valetama|valetatud', 'kingoks', 'luisanud|luiskama', 'laimama|laimatud', 'valetamine', 'varjama|varjanud', 'varjav', 'peitma', 'paljastama', 'varjamine', 'paljastama|paljastanud', 'reetma', 'peitev', 'varjatav', 'varjamata', 'tümitama|tümitanud', 'kuldpagun', 'ratsutav', 'leinarõivas', 'silmalapp', 'krillima', 'valelik', 'poseerinu', 'hõikav', 'uusüritaja', 'kuritarvitama|kuritarvitanud', 'kuritarvitamine', 'ametiseisund', '1993.-1998.', 'kuritarvitanu', 'kuritarvitav', 'teenistushuvi', 'petma', 'kurjasti', 'ameti-seisund', 'manipuleerimine', 'omakasupüüdlikult', 'manipuleeritavus', 'manipuleerija', 'desorienteerima', 'shantazheerima', 'manipuleeritav', 'manipuleeriv', 'manipulatsioon', 'mittemõtlev']


In [134]:
# püstkriipsuga eraldatud sõnad lüüakse lahku, ühtlasi eemaldatakse kordused
vaidl_erim_w2v = set([piece for w in vaidl_erim_w2v for piece in w.split("|")])
valimised_w2v = set([piece for w in valimised_w2v for piece in w.split("|")])
valetamine_w2v = set([piece for w in valetamine_w2v for piece in w.split("|")])
valitsemine_w2v = set([piece for w in valitsemine_w2v for piece in w.split("|")])

In [135]:
print(valetamine_w2v)

{'leinarõivas', 'paljastama', 'silmalapp', 'laimatud', 'valetamine', 'desorienteerima', 'manipuleeritav', 'tümitama', 'ametiseisund', 'valetama', 'omakasupüüdlikult', 'varjamine', 'shantazheerima', 'manipulatsioon', 'peitev', 'kuritarvitama', 'luiskama', 'manipuleeritavus', 'peitma', 'valetatud', 'varjav', 'varjama', 'kuritarvitanu', 'kingoks', 'kuldpagun', 'vassima', 'manipuleerimine', 'luisanud', 'tümitanud', 'petma', 'kuritarvitamine', 'uusüritaja', 'vassinud', 'laimama', 'kuritarvitanud', 'paljastanud', 'poseerinu', 'teenistushuvi', 'hõikav', 'varjatav', 'silmakirjatsema', 'varjamata', 'kuritarvitav', 'varjanud', 'ameti-seisund', 'krillima', 'manipuleeriv', '1993.-1998.', 'valetanud', 'kurjasti', 'reetma', 'valelik', 'mittemõtlev', 'manipuleerija', 'ratsutav'}


In [136]:
# viimaks eemaldatakse leitud sõnade seast sõnad, mis juba märksõnade seas olemas olid
vaidl_erim_w2v = [w for w in vaidl_erim_w2v if w not in vaidl_erim and w not in valimised and w not in valetamine
                  and w not in valitsemine]
valimised_w2v = [w for w in valimised_w2v if w not in vaidl_erim and w not in valimised and w not in valetamine
                and w not in valitsemine]
valetamine_w2v = [w for w in valetamine_w2v if w not in vaidl_erim and w not in valimised and w not in valetamine
                 and w not in valitsemine]
valitsemine_w2v = [w for w in valitsemine_w2v if w not in vaidl_erim and w not in valimised and w not in valetamine
                   and w not in valitsemine]

In [137]:
print(valetamine_w2v)

['leinarõivas', 'paljastama', 'silmalapp', 'laimatud', 'valetamine', 'desorienteerima', 'manipuleeritav', 'tümitama', 'ametiseisund', 'omakasupüüdlikult', 'varjamine', 'shantazheerima', 'manipulatsioon', 'peitev', 'luiskama', 'manipuleeritavus', 'peitma', 'valetatud', 'varjav', 'kuritarvitanu', 'kingoks', 'kuldpagun', 'vassima', 'manipuleerimine', 'luisanud', 'tümitanud', 'petma', 'kuritarvitamine', 'uusüritaja', 'vassinud', 'laimama', 'kuritarvitanud', 'paljastanud', 'poseerinu', 'teenistushuvi', 'hõikav', 'varjatav', 'silmakirjatsema', 'varjamata', 'kuritarvitav', 'varjanud', 'ameti-seisund', 'krillima', 'manipuleeriv', '1993.-1998.', 'valetanud', 'kurjasti', 'reetma', 'valelik', 'mittemõtlev', 'manipuleerija', 'ratsutav']


In [138]:
print(len(vaidl_erim_w2v))
print(len(valimised_w2v))
print(len(valetamine_w2v))
print(len(valitsemine_w2v))

109
71
52
152


### III Andmestikus mitte-esinevate sõnade ja mitte-verbide välja filtreerimine

In [86]:
con = sqlite3.connect("media_data_complete.db")
cur = con.cursor()

In [87]:
sentences = []

In [90]:
# siin võetakse vaid laused, millele hiljem märksõnu peale märgendama hakatakse
for row in cur.execute("SELECT sentence FROM sentences_syntax_analysis"):
    sentence = json_to_text(json_text=row[0])
    sentences.append(sentence)

In [91]:
con.close()

In [97]:
# tühi hulk lausetes esinevate verbide hoiustamiseks
verbs = set()

In [98]:
for sentence in sentences:
    for word in sentence.words:
        if 'V' in word.partofspeech:
            verbs.add(word.lemma[0])

In [139]:
len(verbs)

2502

In [140]:
# mudeli abil leitud sõnade hulgast eemaldatakse sõnad, mis ei esine siinsetes andmetes
vaidl_erim_w2v = [w for w in vaidl_erim_w2v if w in verbs]
valimised_w2v = [w for w in valimised_w2v if w in verbs]
valetamine_w2v = [w for w in valetamine_w2v if w in verbs]
valitsemine_w2v = [w for w in valitsemine_w2v if w in verbs]

In [141]:
print(vaidl_erim_w2v)

['imestama', 'pommitama', 'kritiseerima', 'teotama', 'vastandama', 'piketeerima', 'pommima', 'šantažeerima', 'protesteerima', 'mõnitama', 'arutlema', 'kaitsma', 'tungima', 'pahandama', 'tapma', 'hädaldama', 'virisema', 'argumenteerima', 'eitama', 'kahtlustama', 'kaagutama', 'prääksuma', 'heitlema', 'inisema', 'väitlema', 'kiruma', 'naeruvääristama', 'tulistama', 'kurtma', 'rõõmustama']


In [142]:
print(valimised_w2v)

['alistama', 'lootma', 'võidutsema', 'pürgima', 'kavandama', 'valitud', 'soostuma', 'korraldama', 'tahtma', 'võidetud', 'vastustama', 'plaanima', 'keelduma', 'käskima', 'kohustama', 'nõustuma', 'edestama', 'pooldama', 'soovima']


In [143]:
print(valetamine_w2v)

['paljastama', 'tümitama', 'luiskama', 'peitma', 'vassima', 'petma', 'reetma']


In [144]:
print(valitsemine_w2v)

['määratud', 'reinvesteerima', 'transportima', 'vähendama', 'rahastatud', 'takistama', 'keskendama', 'nentima', 'suunanud', 'investeeritud', 'suunatud', 'väitma', 'halvendama', 'paigaldama', 'arutama', 'möönma', 'paigutatud', 'kanaliseerima', 'sõnama', 'hoolitsema', 'tõdema', 'tõkestama', 'rajama', 'sõltuma', 'märkima', 'toonitama', 'juhtinud', 'kitsendama', 'kontsentreerima', 'pärssima', 'rõhutama', 'kahjustama', 'ehitama', 'ütlema', 'juhitud', 'määratlema']


Kuna leitud sõnade seas on sobivaid palju, lisatakse siin sõnad automaatselt märksõnastikku ning ebasobivad sõnad eemaldatakse või tõstetakse ümber manuaalselt juba CSV-failis.

In [145]:
with open('thematic_keywords.csv', 'a', newline='', encoding='UTF-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=['lemma', 'class'])
    for word in vaidl_erim_w2v:
        writer.writerow({'lemma': "'" + word + "'" + ",", 'class': 'vaidlused ja erimeelsused'})
    for word in valimised_w2v:
        writer.writerow({'lemma': "'" + word + "'" + ",", 'class': 'valimised'})
    for word in valetamine_w2v:
        writer.writerow({'lemma': "'" + word + "'" + ",", 'class': 'valetamine'})
    for word in valitsemine_w2v:
        writer.writerow({'lemma': "'" + word + "'" + ",", 'class': 'valitsemine'})

In [146]:
# enne sobimatute sõnade manuaalset eemaldamist
with open('thematic_keywords.csv', 'r', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [147]:
# enne sobimatute sõnade manuaalset eemaldamist
print(rows)

[{'lemma': 'callable', 'class': 'string'}, {'lemma': "'vastanduma',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'võitlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'häbistama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'vaidlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'nokkima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'protestima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'sõdima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'debateerima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'diskuteerima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'nurisema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'ründama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'süüdistama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'kandideerima',", 'class': 'valimised'}, {'lemma': "'valima',", 'class': 'valimised'}, {'lemma': "'võitma',", 'class': 'valimised'}, {'lemma': "'lubama',", 'class': 'va

In [148]:
# pärast sobimatute sõnade manuaalset eemaldamist või ümber tõstmist
with open('thematic_keywords.csv', 'r', encoding='UTF-8') as csv_file:
    rows = []
    reader = csv.DictReader(csv_file)
    for row in reader:
        rows.append(row)

In [149]:
# pärast sobimatute sõnade manuaalset eemaldamist või ümber tõstmist
print(rows)

[{'lemma': 'callable', 'class': 'string'}, {'lemma': "'vastanduma',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'võitlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'häbistama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'vaidlema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'nokkima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'protestima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'sõdima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'debateerima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'diskuteerima',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'nurisema',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'ründama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'süüdistama',", 'class': 'vaidlused ja erimeelsused'}, {'lemma': "'kandideerima',", 'class': 'valimised'}, {'lemma': "'valima',", 'class': 'valimised'}, {'lemma': "'võitma',", 'class': 'valimised'}, {'lemma': "'lubama',", 'class': 'va